# 自然言語処理２：言語モデル

Python入門では、第７回で文字n-gramを生成し、著者が分からない小説が太宰治のものか宮沢賢治のものかを当てるというミニプロジェクトをやってもらいました。   
今回は、形態素解析結果を用いて、単語n-gramを使った自然言語処理について勉強しましょう。

ここで、Janomeで生成した分かち書き文は単語区切りではなく形態素区切りですが、   
必ずしも言語学的な形態素とは言えない（単語と呼ぶべき）用語も交じっており、   
また、形態素区切りを単語区切りに変換するのは言語学的知識が必要となるので、   
ここではJanomeが出力した形態素を単語と呼び変えることにします。

本教材の作品データは[青空文庫](https://www.aozora.gr.jp/index.html)のものを使用しています。   
ただし、ルビや入力者注、アクセント分解された欧文や編者による注記等は削除しました。   
また、詩のように短い文章から構成されるものをのぞくなど、調整を行っています。


## 1. 準備：分かち書き

言語モデルを学習する前に、解析したい文を分かち書きしましょう。

文章を分かち書きに変換するツールは前の資料で紹介しました。   
ここでは、宮沢賢治の作品147品の本文をjanomeを使って分かち書きし、１つのファイルにまとめた`miyazawa_wakati.txt`を使って処理を行います。   
（このファイルの作り方はわかりますね？本当は皆さんにやっていただきたいのですが、全147作品の形態素解析はそこそこ時間がかかります。）


In [1]:
file = 'text/miyazawa_wakati.txt'

words = [('<BOP> ' + l + ' <EOP>').split() for l in open(file, 'r', encoding='utf-8').readlines()]

# 先頭の2文分を出力してみましょう
print(words[:2])


[['<BOP>', 'その', '明け方', 'の', '空', 'の', '下', '、', 'ひる', 'の', '鳥', 'で', 'も', 'ゆか', 'ない', '高い', 'ところ', 'を', 'するどい', '霜', 'の', 'かけ', 'ら', 'が', '風', 'に', '流さ', 'れ', 'て', 'サラサラ', 'サラサラ', '南', 'の', 'ほう', 'へ', 'とん', 'で', 'ゆき', 'まし', 'た', '。', '<EOP>'], ['<BOP>', 'じつに', 'その', 'かすか', 'な', '音', 'が', '丘', 'の', '上', 'の', '一', '本', 'いちょう', 'の', '木', 'に', '聞こえる', 'くらい', 'すみきっ', 'た', '明け方', 'です', '。', '<EOP>']]


## 2. n-gram生成
### 2.1 自然言語処理用ライブラリNLTKの利用

n = 1のときuni-gram, n = 2のときbi-gram、n = 3のときtri-gramと呼びます。   
宮沢作品集の分かち書きが入っている`words`から、uni-gram, bi-gram, tri-gramをそれぞれ計算してみましょう。   
ここではNLTKのn-gram生成モジュールであるngramsを使います。

In [2]:
from nltk.util import ngrams

text_unigrams = [ngrams(word, 1) for word in words] # uni-gramを計算
text_bigrams = [ngrams(word, 2) for word in words] # bi-gramを計算
text_trigrams = [ngrams(word, 3) for word in words] # tri-gramを計算

# 最初の1文のn-gramを出力してみましょう
print('unigram', [x for x in text_unigrams[0]])
print('bigram',[x for x in text_bigrams[0]])
print('trigram',[x for x in text_trigrams[0]])
    

unigram [('<BOP>',), ('その',), ('明け方',), ('の',), ('空',), ('の',), ('下',), ('、',), ('ひる',), ('の',), ('鳥',), ('で',), ('も',), ('ゆか',), ('ない',), ('高い',), ('ところ',), ('を',), ('するどい',), ('霜',), ('の',), ('かけ',), ('ら',), ('が',), ('風',), ('に',), ('流さ',), ('れ',), ('て',), ('サラサラ',), ('サラサラ',), ('南',), ('の',), ('ほう',), ('へ',), ('とん',), ('で',), ('ゆき',), ('まし',), ('た',), ('。',), ('<EOP>',)]
bigram [('<BOP>', 'その'), ('その', '明け方'), ('明け方', 'の'), ('の', '空'), ('空', 'の'), ('の', '下'), ('下', '、'), ('、', 'ひる'), ('ひる', 'の'), ('の', '鳥'), ('鳥', 'で'), ('で', 'も'), ('も', 'ゆか'), ('ゆか', 'ない'), ('ない', '高い'), ('高い', 'ところ'), ('ところ', 'を'), ('を', 'するどい'), ('するどい', '霜'), ('霜', 'の'), ('の', 'かけ'), ('かけ', 'ら'), ('ら', 'が'), ('が', '風'), ('風', 'に'), ('に', '流さ'), ('流さ', 'れ'), ('れ', 'て'), ('て', 'サラサラ'), ('サラサラ', 'サラサラ'), ('サラサラ', '南'), ('南', 'の'), ('の', 'ほう'), ('ほう', 'へ'), ('へ', 'とん'), ('とん', 'で'), ('で', 'ゆき'), ('ゆき', 'まし'), ('まし', 'た'), ('た', '。'), ('。', '<EOP>')]
trigram [('<BOP>', 'その', '明け方'), ('その', '明け方', 'の'), ('明け方', 'の', '

### 2.2 n-gramの出現回数

nltk.NgramCounterを使って各n-gramが宮沢作品集にそれぞれいくつずつ登場するか数えてみましょう。   
まずは下準備です。

In [3]:
from nltk.util import ngrams
from nltk.lm import NgramCounter

# じつはngramsが返すのはイテレータなので何度も使いまわせないことに注意！
text_unigrams = [ngrams(word, 1) for word in words] # uni-gramを計算
text_bigrams = [ngrams(word, 2) for word in words] # bi-gramを計算
text_trigrams = [ngrams(word, 3) for word in words] # tri-gramを計算

# uni-gram, bi-gram, tri-gramをまとめて一気に出現回数を数える
ngram_counts = NgramCounter(text_unigrams + text_bigrams +text_trigrams) # N-gramの出現回数をカウント


上で計算した結果を使ってさまざまなn-gramの出現回数を表示してみよう。   

ここで、
- `ngram_counts['ジヨバンニ'])`には「ジヨバンニ」の出現回数
- `ngram_counts[['ジヨバンニ']]`には、「ジヨバンニ」に続く単語の出現回数
- `ngram_counts[['ジヨバンニ','は']]`には、「ジヨバンニ は」という２つの単語の後に続く単語の出現回数

が記録されていることに注意してください。

In [4]:
# 「ジヨバンニ」のuni-gram出現回数
print('ジヨバンニ: ' + str(ngram_counts['ジヨバンニ']))

# 「ジヨバンニ」に続く単語の出現回数（bi-gram出現回数）
print('ジヨバンニ')
for word, count in sorted(ngram_counts[['ジヨバンニ']].items(), key=lambda x:x[1], reverse=True):
    print('->\t{:s}: {:d}'.format(word, count))
print()

# 「ジヨバンニ は」に続く単語の出現回数（tri-gram出現回数）
print('ジヨバンニ は')
for word, count in sorted(ngram_counts[['ジヨバンニ','は']].items(), key=lambda x:x[1], reverse=True):
    print('->\t{:s}: {:d}'.format(word, count))


ジヨバンニ: 205
ジヨバンニ
->	は: 123
->	が: 27
->	の: 21
->	も: 10
->	に: 6
->	、: 5
->	さん: 4
->	を: 3
->	たち: 3
->	と: 1
->	や: 1
->	まで: 1

ジヨバンニ は
->	、: 33
->	思は: 7
->	まるで: 5
->	思ひ: 5
->	まつ: 5
->	もう: 4
->	すぐ: 2
->	その: 2
->	何: 2
->	俄: 2
->	窓: 2
->	橋: 2
->	なんだか: 2
->	云: 2
->	また: 2
->	勢: 1
->	手: 1
->	拾: 1
->	おじぎ: 1
->	靴: 1
->	玄: 1
->	立つ: 1
->	高く: 1
->	われ: 1
->	帽子: 1
->	せ: 1
->	なんとも: 1
->	走り: 1
->	ぢ: 1
->	町: 1
->	眼: 1
->	一: 1
->	叫び: 1
->	まだ: 1
->	なぜ: 1
->	どんどん: 1
->	いきなり: 1
->	みんな: 1
->	わくわく: 1
->	かすか: 1
->	それ: 1
->	（: 1
->	胸: 1
->	びつくり: 1
->	困: 1
->	たしかに: 1
->	こんな: 1
->	首: 1
->	坊: 1
->	川下: 1
->	生: 1
->	熱: 1
->	どうしても: 1
->	だんだん: 1
->	もうす: 1
->	あぶなく: 1
->	そつ: 1
->	自分: 1
->	唇: 1
->	力強く: 1
->	叫ん: 1


## 3. 言語モデルの学習

n-gram確率とは、その文書において、直前の$(n-1)$単語が与えられたとき、その後に続く単語の出現確率を表しています。   
たとえば上の例では、「ジヨバンニ は」という２単語が現れたとき、その後に続くのは「、」が３３回です。   
「ジヨバンニ は」の出現回数は123回なので、「ジヨバンニ は」の後に「、」が続く確率（tri-gram確率）は$33/123=0.268293$ということになります。   
これをすべてのn-gramについて計算することを、ここでは「学習」と呼びます。

tri-gram確率を最尤推定法（Maximum Likelihood Estimation)により学習します。
これには少し時間がかかります。

In [11]:
from nltk.lm import Vocabulary
from nltk.lm.models import MLE
from nltk.util import ngrams

# 読み込んだ小説集の語彙（異なり単語）を収集
# Vocabularyは1次元のリストを受け取るが、wordsは2次元のリストなので、
# wordsを内包表記で2次元から1次元に変換してからVocabularyに渡しています
vocab = Vocabulary([item for sublist in words for item in sublist])

# 語彙の一覧を表示させたいなら下4行のコメントを有効にする
num = 0
for v in sorted(vocab.counts):
    print('{:d}\t{:s}'.format(num,v))
    num += 1

print('Vocabulary size: ' + str(len(vocab))) # 語彙サイズ（単語の種類数）

text_trigrams = [ngrams(word, 3) for word in words] # tri-gramを生成

n = 3
lm = MLE(order = n, vocabulary = vocab) # 最尤推定法（Maximum Likelihood Estimation)による統計的n-gram言語モデルの準備
lm.fit(text_trigrams) # 上で生成したtri-gramを使って言語モデルを学習

0	!
1	!……
2	!）
3	"
4	'
5	,
6	,……
7	-
8	------------------------------------------------------------------
9	------------------------------------------------------------------------------------------
10	.
11	.”
12	1000
13	12
14	17
15	2
16	20
17	25
18	288
19	2976
20	30
21	3000
22	4
23	48
24	5500
25	600000
26	62
27	9
28	96
29	;）
30	<BOP>
31	<EOP>
32	Art
33	Astilbe
34	Beethoven
35	Bonan
36	Brownian
37	CZ
38	Capriccioso
39	Casual
40	Conc
41	Cork
42	Dahlia
43	Daniel
44	Defoe
45	Dwarf
46	Egmont
47	Eine
48	Fantasy
49	Fortuny
50	Fox
51	Funeral
52	God
53	Gossan
54	Green
55	H
56	HELL
57	HOUSE
58	Hacienda
59	Herbst
60	Ho
61	Hoff
62	Horadium
63	I
64	Ice
65	In
66	Iwate
67	Josef
68	KeolgKohl
69	KeolgKol
70	LOVE
71	Largo
72	Larix
73	Lento
74	Libido
75	Miss
76	Morgen
77	Morris
78	Nature
79	Nearer
80	Nymbus
81	Nymph
82	Nymphaea
83	Nymphaus
84	O
85	Ora
86	Orade
87	Oscar
88	Overture
89	Pasternack
90	Phantasie
91	Pirr
92	Prrrrr
93	Punkt
94	RESTAURANT
95	Robin
96	Rondo
97	Shitori
98	Sinjoro
99	Sun
100	Super

2181	ぐったがってしばらくくしゃみをするような
2182	ぐったり
2183	ぐってめぐってこゝまで
2184	ぐっと
2185	ぐつた
2186	ぐつたり
2187	ぐつたりする
2188	ぐつたりつかれたやうに
2189	ぐつてあらはれるやうになつて
2190	ぐつと
2191	ぐづぐづしてるんだ
2192	ぐづぐづしてゐるの
2193	ぐづごとさ
2194	ぐづだねえ
2195	ぐづどみんなして
2196	ぐて
2197	ぐとこかも
2198	ぐとすぐどぶんどぶんと
2199	ぐとのさまがえるのけらいになりました
2200	ぐど
2201	ぐな
2202	ぐなぃ
2203	ぐなぃがったが
2204	ぐなぃはんて
2205	ぐなぃもな
2206	ぐなぃもんさ
2207	ぐなぃよだんすじゃ
2208	ぐなぃんぢゃ
2209	ぐなぢやい
2210	ぐなった
2211	ぐなったらど
2212	ぐなってしまたもす
2213	ぐなつたんちやい
2214	ぐなつても
2215	ぐなて
2216	ぐなてよ
2217	ぐなひあがなひし
2218	ぐなるさ
2219	ぐなるような
2220	ぐにいそいでやつてくる
2221	ぐにじぶんのうちへもどって
2222	ぐにすぱりととれた
2223	ぐにそつちへかけて
2224	ぐにつゞいてゐるのでしたから
2225	ぐにゃぐにゃ
2226	ぐにやんとまがりあわててまつすぐを
2227	ぐねの
2228	ぐの
2229	ぐのが
2230	ぐのす
2231	ぐのだべ
2232	ぐのぼれ
2233	ぐはぐ
2234	ぐはしい
2235	ぐはへよ
2236	ぐはをおき
2237	ぐはんて
2238	ぐひけぶるらし
2239	ぐひすが
2240	ぐひすがしきりになき
2241	ぐひすでないよと
2242	ぐひすなんといふ
2243	ぐひすも
2244	ぐひなし
2245	ぐひもあらぬ
2246	ぐへるかの
2247	ぐべ
2248	ぐべがな
2249	ぐべく
2250	ぐべさ
2251	ぐべすさ
2252	ぐべな
2253	ぐまじめになつてうたひました
2254	ぐまちゃ
2255	ぐまでに
2256	ぐまのあしを
2257	ぐみ
2258	ぐみの木
2259	ぐもさない
2260	ぐもそらをはせ
2261	ぐもなぃ
2262	ぐもならなぃで

6681	めらめら
6682	めりめり
6683	めり込み
6684	めん
6685	めんど
6686	めんどう
6687	めんどうくさい
6688	めんどうくさく
6689	も
6690	もい
6691	もう
6692	もうか
6693	もうかっ
6694	もうから
6695	もうかる
6696	もうかん
6697	もうけ
6698	もうける
6699	もうけろ
6700	もうこ
6701	もうさ
6702	もうし
6703	もうしゃ
6704	もうす
6705	もうすぐ
6706	もうそ
6707	もうもうと
6708	もうろく
6709	もう一度
6710	もう少し
6711	もえ
6712	もえる
6713	もがい
6714	もがか
6715	もがき
6716	もぎ
6717	もぎり
6718	もくもく
6719	もぐ
6720	もぐっ
6721	もぐもぐ
6722	もぐり
6723	もぐり込み
6724	もぐり込ん
6725	もげ
6726	もご
6727	もさ
6728	もし
6729	もしか
6730	もしも
6731	もしもし
6732	もしゃもしゃ
6733	もしや
6734	もじ
6735	もじっ
6736	もじもじ
6737	もじゃもじゃ
6738	もすそ
6739	もそ
6740	もそもそ
6741	もた
6742	もたげ
6743	もたせ
6744	もたらさ
6745	もたらし
6746	もたらす
6747	もだえ
6748	もだえる
6749	もだし
6750	もち
6751	もちろん
6752	もっ
6753	もったいぶっ
6754	もって
6755	もってこい
6756	もっと
6757	もっとい
6758	もっとも
6759	もつ
6760	もつとも
6761	もつれ
6762	もつれる
6763	もて
6764	もてる
6765	もと
6766	もとめ
6767	もとより
6768	もどかし
6769	もどかしい
6770	もどかしく
6771	もどき
6772	もどっ
6773	もどり
6774	もどる
6775	もの
6776	ものがたり
6777	ものがたる
6778	ものさし
6779	ものさびしい
6780	ものすごい
6781	ものすさまじい
6782	ものの
6783	もののふ
6784	ものめずらしい
6785	も

10688	内丸
10689	内側
10690	内内
10691	内分
10692	内地
10693	内外
10694	内奥
10695	内容
10696	内密
10697	内川
10698	内庭
10699	内心
10700	内的
10701	内省
10702	内藤
10703	内部
10704	内面
10705	円
10706	円い
10707	円か
10708	円き
10709	円く
10710	円み
10711	円光
10712	円卓
10713	円満寺
10714	円熟
10715	円筒
10716	円舞
10717	円蓋
10718	円通寺
10719	円錐
10720	冊
10721	册
10722	再び
10723	再度
10724	再来
10725	再現
10726	再生
10727	再考
10728	再臨
10729	冒さ
10730	冗談
10731	写
10732	写し
10733	写しもの
10734	写本
10735	写楽
10736	写生
10737	写真
10738	写経
10739	冠
10740	冥
10741	冥加
10742	冬
10743	冴
10744	冴え
10745	冴える
10746	冷
10747	冷え
10748	冷える
10749	冷えれ
10750	冷し
10751	冷た
10752	冷たい
10753	冷たかっ
10754	冷たく
10755	冷め
10756	冷やか
10757	冷やさ
10758	冷やす
10759	冷却
10760	冷汗
10761	冷然
10762	冷笑
10763	冽
10764	凄
10765	凄い
10766	凄く
10767	凄まじい
10768	凄まじく
10769	准
10770	凍
10771	凍え
10772	凍える
10773	凍っ
10774	凍み
10775	凍ら
10776	凍り
10777	凍りつい
10778	凍りつく
10779	凍り付い
10780	凍る
10781	凍れ
10782	凛と
10783	凜
10784	凜々
10785	凝
10786	凝り
10787	凝灰岩
10788	凡そ
10789	処
10790	処々
10791	処さ
10792	処士
10793	処女
10794	処方
10795	処罰
10796	凧
10797	凱旋
1

15180	於け
15181	施さ
15182	施し
15183	施肥
15184	施行
15185	施設
15186	施身大
15187	旁
15188	旅
15189	旅だち
15190	旅だつ
15191	旅人
15192	旅団
15193	旅客
15194	旅程
15195	旅立ち
15196	旅行
15197	旅装
15198	旅費
15199	旋
15200	旋律
15201	旋転
15202	旌
15203	族
15204	旗
15205	旗手
15206	旗雲
15207	既に
15208	既成
15209	日
15210	日々
15211	日ごろ
15212	日だまり
15213	日なた
15214	日の出
15215	日光
15216	日向
15217	日土
15218	日射し
15219	日山
15220	日常
15221	日当り
15222	日影
15223	日数
15224	日日
15225	日昔
15226	日暮れ
15227	日曜
15228	日曜日
15229	日本
15230	日本人
15231	日本海
15232	日本犬
15233	日本語
15234	日永
15235	日照
15236	日照り
15237	日照り雨
15238	日程
15239	日給
15240	日脚
15241	日蔭
15242	日覆
15243	日詰
15244	日誌
15245	日輪
15246	日間
15247	日陰
15248	旦那
15249	旧
15250	旧式
15251	旧暦
15252	旨
15253	早
15254	早々
15255	早い
15256	早う
15257	早かっ
15258	早く
15259	早くから
15260	早口
15261	早春
15262	早見
15263	早足
15264	早速
15265	旬
15266	旭川
15267	旱
15268	旱天
15269	旱害
15270	旱魃
15271	旺
15272	昆布
15273	昆虫
15274	昇
15275	昇っ
15276	昇り
15277	昇る
15278	昇れ
15279	昇汞
15280	昇華
15281	昇降
15282	昌
15283	昌一
15284	明
15285	明い
15286	明か
15287	明き
15288	明く
15289	明け
152

19006	綱
19007	綱取
19008	網
19009	網戸
19010	網打ち
19011	網棚
19012	網膜
19013	綴
19014	綴じ
19015	綴ら
19016	綴り合せ
19017	綺語
19018	綺麗
19019	綾
19020	綿
19021	綿ネル
19022	綿羊
19023	綿雲
19024	緊張
19025	緊那羅
19026	総
19027	総じて
19028	総て
19029	総代
19030	総出
19031	総動員
19032	総合
19033	総掛り
19034	総理
19035	総立ち
19036	総裁
19037	総身
19038	総長
19039	緑
19040	緑玉
19041	緑色
19042	緑茶
19043	緑青
19044	緒
19045	線
19046	線路
19047	締
19048	締金
19049	編
19050	編み
19051	編む
19052	編ん
19053	編棒
19054	緩い
19055	緩ん
19056	緩慢
19057	緯度
19058	練
19059	練っ
19060	練り
19061	練兵
19062	練習
19063	練達
19064	緻密
19065	縁
19066	縁側
19067	縁故
19068	縁起
19069	縁辺
19070	縄
19071	縛
19072	縛っ
19073	縛ら
19074	縛る
19075	縛れ
19076	縞
19077	縞物
19078	縣
19079	縦
19080	縦横
19081	縫
19082	縫っ
19083	縫糸
19084	縮
19085	縮ま
19086	縮まっ
19087	縮まる
19088	縮み
19089	縮れ
19090	縮れ毛
19091	縮尺
19092	縮減
19093	績
19094	繁く
19095	繁っ
19096	繁盛
19097	繃帯
19098	繊細
19099	繊維
19100	繊維素
19101	繋
19102	織っ
19103	織り
19104	織りなし
19105	織る
19106	織れ
19107	織物
19108	繖形花
19109	繞
19110	繩
19111	繩綯
19112	繪
19113	繭
19114	繰
19115	繰っ
19116	繰り
19117	繰り返し
1

Vocabulary size: 22793


## 4. 統計的n-gramの活用
### 4.1 指定された単語列に続く単語を調べよう

宮沢作品において、「ジヨバンニは」と来たら次にはどんな単語が続くでしょうか？    
（「ジヨバンニ は」の後に「、」が続く確率は先ほど計算しましたね。0.268293でした）。  
その確率を計算してみましょう。   

In [6]:
context = ['ジヨバンニ', 'は']
#context = ['カムパネルラ', 'は']
print(context)

# contextに続く単語のリストを獲得
# lm.vocab.lookup(context)は、contextに含まれる単語のうち、vocabに含まれていない
# 単語があったとき、その単語を<UNK>に置き換える
# （UNKはUn-knownのこと。日本語では未知語と呼ぶ）
prob_list = [(word, lm.score(word, context)) for word
            in lm.context_counts(lm.vocab.lookup(context))]
prob_list.sort(key=lambda x: x[1], reverse=True)

sum_prob = 0.0 # 「ジョバンニは」に続く単語のtri-gram確率をすべて足すとどうなるでしょう？
for word, prob in prob_list:
    print('\t{:s}: {:f}'.format(word, prob))
    sum_prob += prob
    
print('ある単語列に続くtri-gram確率をすべて足したら', sum_prob) 
# 結果は1.0になります（誤差があります）
# このtri-gramが、『「ジヨバンニは」という単語列に続く』という条件のもと、
# それに続く単語の出現確率（つまり条件付き確率）になっていることが分かりますね。

['ジヨバンニ', 'は']
	、: 0.268293
	思は: 0.056911
	まるで: 0.040650
	思ひ: 0.040650
	まつ: 0.040650
	もう: 0.032520
	すぐ: 0.016260
	その: 0.016260
	何: 0.016260
	俄: 0.016260
	窓: 0.016260
	橋: 0.016260
	なんだか: 0.016260
	云: 0.016260
	また: 0.016260
	勢: 0.008130
	手: 0.008130
	拾: 0.008130
	おじぎ: 0.008130
	靴: 0.008130
	玄: 0.008130
	立つ: 0.008130
	高く: 0.008130
	われ: 0.008130
	帽子: 0.008130
	せ: 0.008130
	なんとも: 0.008130
	走り: 0.008130
	ぢ: 0.008130
	町: 0.008130
	眼: 0.008130
	一: 0.008130
	叫び: 0.008130
	まだ: 0.008130
	なぜ: 0.008130
	どんどん: 0.008130
	いきなり: 0.008130
	みんな: 0.008130
	わくわく: 0.008130
	かすか: 0.008130
	それ: 0.008130
	（: 0.008130
	胸: 0.008130
	びつくり: 0.008130
	困: 0.008130
	たしかに: 0.008130
	こんな: 0.008130
	首: 0.008130
	坊: 0.008130
	川下: 0.008130
	生: 0.008130
	熱: 0.008130
	どうしても: 0.008130
	だんだん: 0.008130
	もうす: 0.008130
	あぶなく: 0.008130
	そつ: 0.008130
	自分: 0.008130
	唇: 0.008130
	力強く: 0.008130
	叫ん: 0.008130
ある単語列に続くtri-gram確率をすべて足したら 1.0000000000000018


これはn-gram確率です。上でNgramCounterをつかって計算したn-gram出現回数と結果を比べてみてください。   
NgramCounterで数えたのは「回数」ですが、lm(言語モデル）では「確率」になっています。   

### 4.2 ランダム文生成

「ジヨバンニは」に続く文を適当に生成してみましょう。実際にあまり使いどころはないですが、今回の課題の一つですのでやってみてください。  

すでにtri-gram確率を計算しているので、ここでは、直前の2単語だけをみて、次に続く単語の出現確率が0じゃないものをランダムに選んでつないでいきます。   
まさしく人工無能の真骨頂です。

関数lm.generateは、text=seedで指定された単語列を直前の文として、次に続く単語をランダムに選びます。   
このとき、その単語が選ばれる確率は、そのn-gram確率に従います。

In [8]:
### ランダム文生成 ####
# contextから始まる文を生成

# 最初の2単語はいろいろと変えてみましょう
context = ['ジヨバンニ', '、']
#context = ['カムパネルラ', 'は']
print(context)

for i in range(0, 100):
    # contextのうち最後の2単語から次に繋がる確率0じゃない単語をランダムに選ぶ
    w = lm.generate(text_seed=context)
    context.append(w) # 選ばれた単語をcontextに連結
    print(context)
    
    if '。' == w or '<EOP>' == w: # 句点「。」か<EOP>に到達したらそこで終了
        break
   

['ジヨバンニ', '、']
['ジヨバンニ', '、', 'ラツコ']
['ジヨバンニ', '、', 'ラツコ', '裏']
['ジヨバンニ', '、', 'ラツコ', '裏', 'の']
['ジヨバンニ', '、', 'ラツコ', '裏', 'の', '岩穴']
['ジヨバンニ', '、', 'ラツコ', '裏', 'の', '岩穴', 'の']
['ジヨバンニ', '、', 'ラツコ', '裏', 'の', '岩穴', 'の', 'ところ']
['ジヨバンニ', '、', 'ラツコ', '裏', 'の', '岩穴', 'の', 'ところ', 'へ']
['ジヨバンニ', '、', 'ラツコ', '裏', 'の', '岩穴', 'の', 'ところ', 'へ', '行']
['ジヨバンニ', '、', 'ラツコ', '裏', 'の', '岩穴', 'の', 'ところ', 'へ', '行', 'つ']
['ジヨバンニ', '、', 'ラツコ', '裏', 'の', '岩穴', 'の', 'ところ', 'へ', '行', 'つ', 'た']
['ジヨバンニ', '、', 'ラツコ', '裏', 'の', '岩穴', 'の', 'ところ', 'へ', '行', 'つ', 'た', 'か']
['ジヨバンニ', '、', 'ラツコ', '裏', 'の', '岩穴', 'の', 'ところ', 'へ', '行', 'つ', 'た', 'か', 'ただ']
['ジヨバンニ', '、', 'ラツコ', '裏', 'の', '岩穴', 'の', 'ところ', 'へ', '行', 'つ', 'た', 'か', 'ただ', '蒼']
['ジヨバンニ', '、', 'ラツコ', '裏', 'の', '岩穴', 'の', 'ところ', 'へ', '行', 'つ', 'た', 'か', 'ただ', '蒼', '黝']
['ジヨバンニ', '、', 'ラツコ', '裏', 'の', '岩穴', 'の', 'ところ', 'へ', '行', 'つ', 'た', 'か', 'ただ', '蒼', '黝', 'く']
['ジヨバンニ', '、', 'ラツコ', '裏', 'の', '岩穴', 'の', 'ところ', 'へ', '行', 'つ', 'た', 'か', 'ただ', '蒼', '黝', 'く'

## 5. 文の生起確率$P(W)$の算出

宮沢賢治の小説集をコーパスとして学習した統計的tri-gramを使って、与えられた文（単語列）がどの程度『宮沢賢治らしい』かを推定してみましょう。   
これは、n-gram確率を

$$p(w_i|w_{i-n+1},...,w_{i-1})$$

としたとき、入力文$W=({w_0, w_1, ..., w_n})$に対して、文の生起確率

$$P(W)=\Pi_{i=0}^{n+1}p(w_i|w_{i-n+1},...,w_{i-1})$$

の計算をすることを意味しています(ただし、$w_i$はその文の$i$番目の単語を表し、$w_0$のとき文頭、$w_{n+1}$のとき文末記号を表す）。   
詳しくは授業でスライドを使って説明します。

In [10]:
### 入力文がどの程度宮沢賢治らしい文章かを判定してみよう ###

# どちらの文のほうが生起確率が高いでしょうか？構成する単語はどちらも同じです。
line = 'ジヨバンニ は 何 げ なく 答え まし た 。'
#line = '何 げ なく ジヨバンニ は 答え まし た 。'

words2 = line.split() # 空白で区切ってリストに代入します

n = 3 # lm (言語モデル) はtri-gramで学習しているので、ここではn=3を指定
probability = 1.0 # 始め確率は1.0に初期化
for ngram in ngrams(words2, n):
    # 2単語の後に3単語目が続く確率をひたすら計算
    prob = lm.score(lm.vocab.lookup(ngram[-1]),
                    lm.vocab.lookup(ngram[:-1]))
    print(ngram[:-1], ngram[-1], ':\t', prob)
    print()
    
    # 確率が0(すなわち、そのような文字の連結は宮沢作品に一度も現れない)の場合は
    # そのtri-gramの生起確率は0になってしまう。それをProbabilityに掛けると、
    # たとえ他のtri-gramが頻出するものであっても0になるので、
    # ここでは0のときは微小な値をprobに代入する
    prob = max(prob, 1e-8)
    
    # tri-gramの生起確率をかけ合わせていく
    probability *= prob
    
print(probability)

('ジヨバンニ', 'は') 何 :	 0.016260162601626018

('は', '何') げ :	 0.006493506493506494

('何', 'げ') なく :	 1.0

('げ', 'なく') 答え :	 0.3333333333333333

('なく', '答え') まし :	 1.0

('答え', 'まし') た :	 1.0

('まし', 'た') 。 :	 0.8878514702725907

3.124807201888539e-05


- 「何 げ なく ジヨバンニ は 答え まし た 。」の出現確率は6.658886027044431e-25
- 「ジヨバンニ は 何 げ なく 答え まし た 。」の出現確率は3.124807201888539e-05

ということで、後者の方がそれらしい文であることがわかります。